In [107]:
import folium
import folium.plugins
import pymongo
import pandas as pd
from folium.features import *
from math import sin, cos, sqrt, atan2, radians

In [108]:
pd.options.display.max_rows = 999

In [109]:
client = pymongo.MongoClient("localhost", 27017)
coll = client.col_en_datos.cities

In [110]:
#db.getCollection('cities').find({
#   population: {$gt:10000},
#   location: {
#     $near: {
#       $geometry: {
#          type: "Point" ,
#          coordinates: [ -74.0721 , 4.7110 ]
#       }
#     }
#   }
#})

In [111]:
def get_nearest_city(coor):
    lat = coor[1]
    lon = coor[0]
    res = coll.find({
        'population': {'$gt':5000},
        'location': {
            '$near': {
                '$geometry': {
                    'type': "Point" ,
                    'coordinates': [ lon , lat ]
                }
            }
        }
    })
    return res[1]

In [112]:
def get_distance(p0, p1):
    R = 6373.0

    lat1 = radians(p0[1])
    lon1 = radians(p0[0])
    lat2 = radians(p1[1])
    lon2 = radians(p1[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [113]:
cities = coll.find({'population': {'$gt':5000}, 'country':'co'})
#cities = coll.find({'population': {'$gt':100000}})
#cities = coll.find({'population': {'$gt':10000}})

In [114]:
distances_df = []
for city in cities:
    nearest_city = get_nearest_city(city['location']['coordinates'])
    d = {}
    d['city'] = city['city']
    d['city_coor'] = city['location']['coordinates']
    d['nearest_city'] = nearest_city['city']
    d['nearest_city_coor'] = nearest_city['location']['coordinates']
    d['distance'] = get_distance(city['location']['coordinates'], nearest_city['location']['coordinates'])
    distances_df.append(d)
distances_df = pd.DataFrame(distances_df)

In [115]:
top_5 = distances_df.sort_values('distance', ascending=False).head(5)

In [116]:
top_5

,city,city_coor,distance,nearest_city,nearest_city_coor
246,Mitú,"[-70.173333, 1.198333]",314.528032,San José del Guaviare,"[-72.645914, 2.572858]"
189,Inírida,"[-67.923889, 3.865278]",202.808878,Puerto Ayacucho,"[-67.62361109999999, 5.6638889]"
312,Puerto Leguízamo,"[-74.781886, -0.193368]",170.122551,Cartagena del Chairá,"[-74.842886, 1.334884]"
362,San Andrés,"[-81.700556, 12.584722]",152.082628,Corn Island,"[-83.0333333, 12.166666699999999]"
38,Bajo Baudó,"[-77.365975, 4.953336]",79.000410,Istmina,"[-76.683971, 5.160538]"


In [117]:
class DivIcon(MacroElement):
    def __init__(self, html='', size=(30,30), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)

In [118]:
m = folium.Map([4.7110, -74.0721],
                tiles='https://korona.geog.uni-heidelberg.de/tiles/roadsg/x={x}&y={y}&z={z}',
                attr='&copy; <a href="http://osm.org/copyright">OpenStreetMap</a> contributors',
                zoom_start=5)

for city_index, city in enumerate(top_5.iterrows()):
    #City
    coor_city = city[1]['city_coor']
    folium.map.Marker(
        [coor_city[1],coor_city[0]]).add_to(m)
    folium.map.Marker(
        [coor_city[1],coor_city[0]],
        icon=DivIcon(
            size=(150,36),
            anchor=(150,0),
            html="%s. %s" % ((city_index + 1), city[1]['city']),
            style="""
                font-size:20px;
                background-color: transparent;
                border-color: transparent;
                text-align: right;
                """
            )
        ).add_to(m)
    
    #Nearest city
    coor_n_city = city[1]['nearest_city_coor']
    folium.map.Marker(
        [coor_n_city[1],coor_n_city[0]]).add_to(m)
    folium.map.Marker(
        [coor_n_city[1],coor_n_city[0]],
        icon=DivIcon(
            size=(150,36),
            anchor=(150,0),
            html="%s (%s km)" % (city[1]['nearest_city'], int(city[1]['distance'])),
            style="""
                font-size:15px;
                background-color: transparent;
                border-color: transparent;
                text-align: right;
                """
            )
        ).add_to(m)
    
    folium.PolyLine([[coor_city[1],coor_city[0]], [coor_n_city[1],coor_n_city[0]]], color="red", weight=2.5, opacity=1).add_to(m)

m
#m.save('/tmp/map.html')